In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from  PIL import Image
import numpy as np
from tensorflow.keras import Input, Model
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import split_dataset
import tensorflow as tf
from tensorflow import keras

RSEED = 42
dataset_path = '../data/images/'

In [2]:


#import the data

data = []

# Specify the path to your dataset
dataset_path = '../data/images/'

# Iterate through each plant folder
for plant_class in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, plant_class)
    
    # Iterate through each image in the plant folder
    for image_file in os.listdir(class_path):
        image_path = os.path.join(class_path, image_file)
        data.append({'Image_Path': image_path, 'Class': plant_class})

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

df

,Image_Path,Class
0,../data/images/Tomato___spider_mites/fa05ccd2-...,Tomato___spider_mites
1,../data/images/Tomato___spider_mites/fd3b6292-...,Tomato___spider_mites
2,../data/images/Tomato___spider_mites/82306885-...,Tomato___spider_mites
3,../data/images/Tomato___spider_mites/9c7f0991-...,Tomato___spider_mites
4,../data/images/Tomato___spider_mites/d7c5cad4-...,Tomato___spider_mites
...,...,...
95863,../data/images/Soybean___healthy/57c18b39-2a33...,Soybean___healthy
95864,../data/images/Soybean___healthy/4fdc663e-a8ea...,Soybean___healthy
95865,../data/images/Soybean___healthy/df807f13-078b...,Soybean___healthy
95866,../data/images/Soybean___healthy/60bf9858-951a...,Soybean___healthy


In [3]:
def rename_subfolder(parent_dir, subfolder_name, new_name):
    subfolder_path = os.path.join(parent_dir, subfolder_name)
    new_path = os.path.join(parent_dir, new_name)

    # Check if the subfolder exists
    if os.path.exists(subfolder_path):
        # Rename the subfolder
        os.rename(subfolder_path, new_path)
        print(f"{subfolder_name} renamed to {new_name} successfully.")
    else:
        print(f"Subfolder {subfolder_name} doesn't exist in {parent_dir}.")

# Example usage
parent_dir = "../data/images/"
subfolder_name = "Grape_leaf_blight"
new_name = "Grape___leaf_blight"

rename_subfolder(parent_dir, subfolder_name, new_name)

Subfolder Grape_leaf_blight doesn't exist in ../data/images/.


In [4]:
# Extract plant types from class names
df['plant_type'] = df['Class'].apply(lambda x: x.split('___')[0])

# Count the occurrences of each plant type
plant_type_counts = df['plant_type'].value_counts()

# Display the count of each plant type
print(plant_type_counts)

plant_type
Cassava        21397
Tomato         18160
Potato          8552
Apple           6542
Grape           6232
Rice            5704
Orange          5507
Soybean         5090
Corn            3852
Peach           2657
Sugercane       2521
Bell_pepper     2475
Cherry          1906
Squash          1835
Strawberry      1565
Blueberry       1502
Raspberry        371
Name: count, dtype: int64


In [5]:
# Extract disease types from class names
df['disease_type'] = df['Class'].apply(lambda x: x.split('___')[1])

# Count the occurrences of each disease type
disease_type_counts = df['disease_type'].value_counts()

# Display the count of each disease type
print(disease_type_counts)

disease_type
healthy                 21615
mosaic_disease          13158
citrus_greening          5507
bacterial_spot           5421
leaf_curl                5357
late_blight              3929
early_blight             3628
powdery_mildew           2887
bacterial_blight         2671
green_mottle             2386
black_measles            2271
brown_streak_disease     2189
brown_spot               1815
black_rot                1801
septoria_leaf_spot       1771
rust                     1755
spider_mites             1676
blast                    1440
target_spot              1404
scab                     1222
common_rust              1192
leaf_scorch              1109
tungro                   1080
isariopsis_leaf_spot     1076
northern_leaf_blight      985
leaf_mold                 952
pests                     611
bacterial_wilt            569
virus                     532
red_rot                   518
gray_leaf_spot            513
yellow_leaf               505
mosaic                    4

In [6]:

# Merge the values
df['disease_type'].replace(['mosaic_disease', 'mosaic_virus'], 'mosaic_disease', inplace=True)


In [7]:
disease_type_counts = df['disease_type'].value_counts()

# Display the count of each disease type
print(disease_type_counts)

disease_type
healthy                 21615
mosaic_disease          13531
citrus_greening          5507
bacterial_spot           5421
leaf_curl                5357
late_blight              3929
early_blight             3628
powdery_mildew           2887
bacterial_blight         2671
green_mottle             2386
black_measles            2271
brown_streak_disease     2189
brown_spot               1815
black_rot                1801
septoria_leaf_spot       1771
rust                     1755
spider_mites             1676
blast                    1440
target_spot              1404
scab                     1222
common_rust              1192
leaf_scorch              1109
tungro                   1080
isariopsis_leaf_spot     1076
northern_leaf_blight      985
leaf_mold                 952
pests                     611
bacterial_wilt            569
virus                     532
red_rot                   518
gray_leaf_spot            513
yellow_leaf               505
mosaic                    4